In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import GridSearchCV


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### **Load Data**

In [2]:
!ls '/content/drive/My Drive/Colab Notebooks/Fraud detection'
fraud_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fraud detection/imbalancedFraudDF.csv')
fraud_data

'Fraud Detection.ipynb'   imbalancedFraudDF.csv   IpAddress_to_Country.csv


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0
...,...,...,...,...,...,...,...,...,...,...,...
138371,185619,2015-01-09 02:25:03,2015-01-09 02:25:04,30,AVOVPRJNLOLWD,SEO,IE,M,35,8.031690e+08,1
138372,144031,2015-07-07 15:29:10,2015-08-20 01:48:50,67,YLHYGZVPCLHLH,Ads,FireFox,M,23,3.572177e+09,1
138373,58899,2015-04-24 15:28:22,2015-07-16 01:53:25,54,ORSHWNJVUGJFE,Direct,Safari,M,23,1.374693e+08,1
138374,120669,2015-04-14 13:53:32,2015-05-18 23:52:34,31,HNQOJFPCMBRQU,SEO,Chrome,M,31,7.087370e+08,1


In [3]:
ipToCountry = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fraud detection/IpAddress_to_Country.csv')
ipToCountry

,lower_bound_ip_address,upper_bound_ip_address,country
0,1.677722e+07,16777471,Australia
1,1.677747e+07,16777727,China
2,1.677773e+07,16778239,China
3,1.677824e+07,16779263,Australia
4,1.677926e+07,16781311,China
...,...,...,...
138841,3.758092e+09,3758093311,Hong Kong
138842,3.758093e+09,3758094335,India
138843,3.758095e+09,3758095871,China
138844,3.758096e+09,3758096127,Singapore


### **Data Exploration**

In [4]:
fraud_data.dtypes

user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object

In [5]:
fraud_data['class'].value_counts()

0    136961
1      1415
Name: class, dtype: int64

In [6]:
fraud_data.isna().sum()

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

combine contury into fraud_data

In [0]:
countries = []
for i in range(len(fraud_data)):
  ip_address = fraud_data.loc[i,'ip_address']

  tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
  if len(tmp) == 1:
    countries.append(tmp['country'].values[0])
  else:
    countries.append('NA')

fraud_data['country'] = countries
                    
                  
                    

In [9]:
ip_address = fraud_data.loc[8,'ip_address']
tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address)&
                  (ipToCountry['upper_bound_ip_address'] >= ip_address)]
print(tmp)

      lower_bound_ip_address  upper_bound_ip_address        country
1017             335544320.0               352321535  United States


### **Feature Engineering**

In [10]:
num_of_user = fraud_data.groupby('user_id').sum()
num_of_transaction = pd.notnull(fraud_data['purchase_value']).sum()
print('There are total {} unique user in this dataset'.format(num_of_user.shape[0]))
print('There are total {} transactions in this dataset'.format(num_of_transaction))

There are total 138376 unique user in this dataset
There are total 138376 transactions in this dataset


In [0]:
fraud_data['interval_after_signup'] = (pd.to_datetime(fraud_data['purchase_time']) - pd.to_datetime(fraud_data['signup_time'])).dt.total_seconds()

fraud_data['signup_day_of_year'] = pd.DatetimeIndex(fraud_data['signup_time']).dayofyear
fraud_data['purchase_day_of_year'] = pd.DatetimeIndex(fraud_data['purchase_time']).dayofyear


#bed time transaction / signup
fraud_data['signup_seconds_of_day'] = pd.DatetimeIndex(fraud_data['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['signup_time']).hour

fraud_data['puchase_seconds_of_day'] =  pd.DatetimeIndex(fraud_data['purchase_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['purchase_time']).hour

fraud_data = fraud_data.drop(['user_id','signup_time','purchase_time'],axis=1)

In [12]:
fraud_data.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_day_of_year,purchase_day_of_year,signup_seconds_of_day,puchase_seconds_of_day
0,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,55,108,82549,10031
1,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,158,159,74390,5934
2,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,492085.0,118,124,76405,50090
3,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,202,252,25792,67253
4,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,Canada,4240931.0,141,190,21783,29114


In [13]:
fraud_data['source'].value_counts()

SEO       55766
Ads       54913
Direct    27697
Name: source, dtype: int64

# Q: What now? train/test split? convert categorical features e.g. device_id to numericals? or SMOTE sampling?


In [0]:
# what is SMOTE?
# What if convert source column by OHE now?
# it sneak peek the levels of this column in test data, 
# what if there is another new level "newsPaper" in test, but not in train?   Info leak, know ahead of time


In [15]:
y = fraud_data['class']
X = fraud_data.drop(['class'],axis=1)

# train test split
X_train,X_test, y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)

X_train.shape: (110700, 13)
y_train.shape: (110700,)


In [16]:
X_train.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,country,interval_after_signup,signup_day_of_year,purchase_day_of_year,signup_seconds_of_day,puchase_seconds_of_day
35821,29,WXDZXCSBXXWBD,Direct,FireFox,M,27,3.476950e+09,United States,7923474.0,86,177,7579,68653
11269,23,LICJGQYUYLEDM,Ads,Chrome,F,32,4.106570e+09,NA,6175612.0,33,104,3772,44984
54561,29,IOATDABEDLLAQ,SEO,FireFox,M,28,3.320175e+09,South Africa,9638429.0,78,190,48913,10542
57092,26,OZNTXPRPVXCVR,SEO,Safari,M,42,1.592338e+08,United States,2497028.0,78,107,85593,77021
33517,55,PZZXUPBIZTGJW,SEO,IE,M,18,1.099134e+09,United States,3211461.0,1,38,27776,42437


### **Convert categorital features with high cadinality to numericals**
#converting needs to be done after split


In [17]:
X_train['country'].value_counts(ascending=True)

Burundi                               1
Cape Verde                            1
British Indian Ocean Territory        1
Madagascar                            1
Saint Martin                          1
                                  ...  
United Kingdom                     3273
Japan                              5353
China                              8887
NA                                16244
United States                     42328
Name: country, Length: 179, dtype: int64

In [0]:
X_train = pd.get_dummies(X_train, columns=['source','browser']) # do not need to drop these two columns since auto dropped by get_dummies
X_train['sex'] = (X_train.sex =='M').astype(int)

# the more a device is shared, the more suspicous
X_train_device_id_mapping = X_train.device_id.value_counts(dropna=False)
X_train['n_dev_shared'] = X_train.device_id.map(X_train_device_id_mapping)

# the more a ip is shared, the more suspicious 
X_train_ip_address_mapping = X_train.ip_address.value_counts(dropna=False)
X_train['n_ip_shared'] = X_train.ip_address.map(X_train_ip_address_mapping)

# the less visit from a country, the more suspicious
X_train_country_mapping = X_train.country.value_counts(dropna=False)
X_train['n_country_shared'] = X_train.country.map(X_train_country_mapping)#lots of NAs in country column, #without dropna=False will produce nan in this col


#把country 用出现次数代替，坏处就是 相同country里面体现不出任何区别

X_train = X_train.drop(['device_id','ip_address','country'], axis=1)

In [0]:
# same thing for test set

X_test = pd.get_dummies(X_test, columns=['source', 'browser'])
X_test['sex'] = (X_test.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)



# if the levels/values of the column in train and test data are pretty much the same, e.g. country, 
# then we should apply the above 3 X_train_mappings (generated from X_train) on X_test, 
# rather than using the new mapping generated from X_test
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/target-encoding.html
# the more a device is shared, the more suspicious
# X_test['n_dev_shared'] = X_test.device_id.map(X_train_device_id_mapping)

# the more a ip is shared, the more suspicious
# X_test['n_ip_shared'] = X_test.ip_address.map(X_train_ip_address_mapping)

# the less visit from a country, the more suspicious
# X_test['n_country_shared'] = X_test.country.map(X_train_country_mapping)

#but here device_id, ip_address has few overlap between train and test, if we apply the above 
# 3 X_train_mappings (generated from X_train) on X_test, then most of the X_test will be NaN


In [20]:
X_train.head(20)

,purchase_value,sex,age,interval_after_signup,signup_day_of_year,purchase_day_of_year,signup_seconds_of_day,puchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,n_dev_shared,n_ip_shared,n_country_shared
35821,29,1,27,7923474.0,86,177,7579,68653,0,1,0,0,1,0,0,0,1,1,42328
11269,23,0,32,6175612.0,33,104,3772,44984,1,0,0,1,0,0,0,0,1,1,16244
54561,29,1,28,9638429.0,78,190,48913,10542,0,0,1,0,1,0,0,0,1,1,619
57092,26,1,42,2497028.0,78,107,85593,77021,0,0,1,0,0,0,0,1,1,1,42328
33517,55,1,18,3211461.0,1,38,27776,42437,0,0,1,0,0,1,0,0,1,1,42328
18842,37,1,20,5688502.0,9,75,42950,29052,0,0,1,0,1,0,0,0,1,1,2124
94173,47,1,45,7642679.0,88,176,39835,79314,0,0,1,0,1,0,0,0,1,1,42328
29820,11,0,31,4812918.0,159,215,53632,28150,0,1,0,0,0,1,0,0,1,1,1449
125445,9,1,32,7407697.0,183,269,58973,36270,1,0,0,0,1,0,0,0,1,1,796
6314,67,1,29,9241810.0,29,136,13683,10693,1,0,0,1,0,0,0,0,1,1,42328


# Q: What is the difference between normalization and standardization?


In [0]:
# # normalize (min-max) to [0,1], standardize(StandardScaler) to normal, mu=0,var = 1 can < 0, so we do normalize here

# needs to be brought to the same scale for models like LR with regularization(that are not tree based)

#Compute the train minimum and maximum to be used for later scaling:


scaler = preprocessing.MinMaxScaler().fit(X_train[['n_dev_shared','n_ip_shared','n_country_shared']])

X_train[['n_dev_shared','n_ip_shared','n_country_shared']] = scaler.transform(X_train[['n_dev_shared','n_ip_shared','n_country_shared']])
X_test[['n_dev_shared','n_ip_shared','n_country_shared']] = scaler.transform(X_test[['n_dev_shared','n_ip_shared','n_country_shared']])


In [22]:
X_train.n_dev_shared.value_counts(dropna=False)

0.00    105418
0.25      4796
0.50       339
0.75       112
1.00        35
Name: n_dev_shared, dtype: int64

In [23]:
X_test.n_dev_shared.value_counts(dropna=False)

0.00    27361
0.25      308
0.75        4
0.50        3
Name: n_dev_shared, dtype: int64

# Simple LogisticRegression model

In [0]:
lr = LogisticRegression()

lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)

In [25]:
cm = metrics.confusion_matrix(y_test,y_pred)
cmDF = pd.DataFrame(cm, columns=['pred_0','pred_1'],index=['true_0','true_1'])
cmDF

,pred_0,pred_1
true_0,27387,0
true_1,289,0


# Simple RF model

In [26]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_train,y_train)

# make prediction 0/1 in test set
predicted = rf.predict(X_test)
# generate probability
probs = rf.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted)))#string to int

accuracy_score is: : 0.9953750541985836
roc_auc_score is: : 0.783358810781212
f1_score is: : 0.7155555555555556


In [27]:
print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0

confusion_matrix is: 
        pred_0  pred_1
true_0   27387       0
true_1     128     161
recall = 0.5570934256055363
precision = 1.0


### ***SMOTE Sampling***

In [0]:

#Wrong way to sampling: smote before split train/test, then test data does not reflect true distribution in reality, 
#and “blend” information from the test set into the training of the model. overfit! think about the case of simple 
#oversampling (where I just duplicate observations). If I upsample a dataset before splitting it into a train and 
#validation set, I could end up with the same observation in both datasets

#https://imbalanced-learn.org/en/stable/install.html

# Install
# imbalanced-learn is currently available on the PyPi’s reporitories and you can install it via pip:

# pip install -U imbalanced-learn

#oversampling on only the training data, the right way! 
#sampling_strategy = number of samples in the majority class over the number of samples in the minority class after resampling

In [29]:
smote = SMOTE(random_state =42)
X_train_sm, y_train_sm = smote.fit_sample(X_train,y_train)

unique, counts = np.unique(y_train_sm, return_counts=True)

print(np.asarray((unique,counts)).T)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


[[     0 109574]
 [     1 109574]]


RF model using SMOTED training data

In [30]:
rf_smote = rf.fit(X_train_sm, y_train_sm)

# make prediction on test data

predicted_sm = rf_smote.predict(X_test)

# generate probability

probs_sm = rf_smote.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
print("%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
print("%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm)))#string to int

accuracy_score_sm is: : 0.9951943922532158
roc_auc_score_sm is: : 0.7781941473760123
f1_score_sm is: : 0.7037861915367483


In [31]:

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1]))
print('precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1]))

confusion_matrix_sm is: 
        pred_0  pred_1
true_0   27385       2
true_1     131     158
recall_sm = 0.5467128027681661
precision_sm = 0.9875


### **Parameter tuning by GridSearchCV**

## Eval metrics for GridSearchCV over all fits upon combination of parameters and cv

In [0]:
# Evaluation metrics to be calculated for each combination of parameters and CV
scores = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'f1_score':make_scorer(f1_score, pos_label=1)
}

In [0]:
def grid_search_wrapper(model, parameters, refit_score = 'f1_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """

    grid_search = GridSearchCV(model, parameters, scoring=scores, refit=refit_score,
                           cv=3, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # make predictions
    y_pred = grid_search.predict(X_test)
    y_prob = grid_search.predict_proba(X_test)[:,1]


    print("Best parameters for {}".format(refit_score))
    print(grid_search.best_params_)


    # confusion matrix on the test data
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    cm = confusion_matrix(y_test, y_pred)
    cmDF = pd.DataFrame(cm, columns=['pred_0','pred_1'], index=['true_0','true_1'])
    print(cmDF)

    print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, y_pred)))#string to int

    print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
    print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))
    return grid_search
    

Optimizing on F1 score on LR

In [34]:
# C: inverse of regularization strength, smaller values specify stronger regularization

LRGrid = {'C': np.logspace(-2,2,5), 'penalty': ['l1','l2']} #l1 lasso l2 ridge

logRegModel = LogisticRegression(random_state=0)

grid_search_LR_f1 = grid_search_wrapper(logRegModel, LRGrid, refit_score = 'f1_score')

Best parameters for f1_score
{'C': 0.01, 'penalty': 'l2'}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27387       0
true_1     289       0
	roc_auc_score is: : 0.7674228029538931
	f1_score is: : 0.0
recall = 0.0
precision = nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


Optimizing on F1 score on RF

In [0]:
parameters = {
    'max_depth': [None, 5, 15],
    'n_estimators': [10,150],
    'class_weight': [{0:1,1:w} for w in [0.2,1,100]]
}

clf = RandomForestClassifier(random_state=42)



In [36]:
grid_search_rf_f1 = grid_search_wrapper(clf, parameters,refit_score='f1_score')

Best parameters for f1_score
{'class_weight': {0: 1, 1: 100}, 'max_depth': None, 'n_estimators': 150}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27387       0
true_1     128     161
	roc_auc_score is: : 0.7845223209102189
	f1_score is: : 0.7155555555555556
recall = 0.5570934256055363
precision = 1.0


In [37]:
best_rf_model_f1 = grid_search_rf_f1.best_estimator_
best_rf_model_f1

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 100}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=150, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)

In [38]:
results_f1 = pd.DataFrame(grid_search_rf_f1.cv_results_)
results_sortf1 = results_f1.sort_values(by = 'mean_test_f1_score', ascending=False)
results_sortf1[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()


,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
13,1.000,0.513,0.678,1.0,1.000,1.000,None,"{0: 1, 1: 100}",150
0,0.998,0.513,0.678,1.0,0.869,0.930,None,"{0: 1, 1: 0.2}",10
7,0.998,0.513,0.678,1.0,1.000,1.000,None,"{0: 1, 1: 1}",150
11,0.998,0.513,0.678,1.0,0.556,0.715,15,"{0: 1, 1: 1}",150
1,0.998,0.513,0.678,1.0,1.000,1.000,None,"{0: 1, 1: 0.2}",150


Optimizing recall score on RF model


In [39]:
grid_search_rf_recall = grid_search_wrapper(clf, parameters, refit_score='recall_score')


Best parameters for recall_score
{'class_weight': {0: 1, 1: 100}, 'max_depth': 5, 'n_estimators': 150}

Confusion matrix of Random Forest optimized for recall_score on the test data:
        pred_0  pred_1
true_0   27166     221
true_1     125     164
	roc_auc_score is: : 0.7996019630458874
	f1_score is: : 0.48664688427299707
recall = 0.5674740484429066
precision = 0.42597402597402595


In [40]:
best_rf_model_recall = grid_search_rf_recall.best_estimator_
best_rf_model_recall

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 100}, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=150, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)

In [0]:
# make predictions on the test set
predictedBest_recall = best_rf_model_recall.predict(X_test)


# generate class probability
probsBest_recall = best_rf_model_recall.predict_proba(X_test)


In [42]:
results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)
results_sortrecall = results_recall.sort_values(by= 'mean_test_recall_score', ascending = False)
results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()


,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
15,0.167,0.611,0.262,0.173,0.633,0.272,5,"{0: 1, 1: 100}",150
14,0.218,0.578,0.310,0.230,0.603,0.326,5,"{0: 1, 1: 100}",10
0,0.998,0.513,0.678,1.000,0.869,0.930,None,"{0: 1, 1: 0.2}",10
7,0.998,0.513,0.678,1.000,1.000,1.000,None,"{0: 1, 1: 1}",150
13,1.000,0.513,0.678,1.000,1.000,1.000,None,"{0: 1, 1: 100}",150


### **Task:**

In [43]:
trainDF = pd.concat([X_train, y_train], axis=1)
pd.crosstab(trainDF['n_dev_shared'], trainDF['class'])

class,0,1
n_dev_shared,,
0.00,104952,466
0.25,4428,368
0.50,153,186
0.75,33,79
1.00,8,27


In [44]:
X_train

,purchase_value,sex,age,interval_after_signup,signup_day_of_year,purchase_day_of_year,signup_seconds_of_day,puchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,n_dev_shared,n_ip_shared,n_country_shared
35821,29,1,27,7923474.0,86,177,7579,68653,0,1,0,0,1,0,0,0,0.0,0.0,1.000000
11269,23,0,32,6175612.0,33,104,3772,44984,1,0,0,1,0,0,0,0,0.0,0.0,0.383750
54561,29,1,28,9638429.0,78,190,48913,10542,0,0,1,0,1,0,0,0,0.0,0.0,0.014601
57092,26,1,42,2497028.0,78,107,85593,77021,0,0,1,0,0,0,0,1,0.0,0.0,1.000000
33517,55,1,18,3211461.0,1,38,27776,42437,0,0,1,0,0,1,0,0,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,31,1,54,1632961.0,96,114,5301,83062,1,0,0,1,0,0,0,0,0.0,0.0,0.209937
119879,35,0,23,3236817.0,87,124,45919,85936,0,0,1,0,0,0,0,1,0.0,0.0,0.051835
103694,16,1,33,6398156.0,215,289,59121,63677,1,0,0,1,0,0,0,0,0.0,0.0,0.018144
131932,75,1,44,8737289.0,212,313,63571,74460,1,0,0,0,0,1,0,0,0.0,0.0,0.077303


In [45]:
X_test

,purchase_value,sex,age,interval_after_signup,signup_day_of_year,purchase_day_of_year,signup_seconds_of_day,puchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,n_dev_shared,n_ip_shared,n_country_shared
135822,77,0,34,4318816.0,89,139,7805,6621,0,0,1,1,0,0,0,0,0.25,0.0,0.052496
45145,57,0,35,7375746.0,183,268,36438,68184,0,0,1,1,0,0,0,0,0.00,0.0,0.012663
10039,26,1,30,6337569.0,165,238,2367,32736,0,0,1,1,0,0,0,0,0.00,0.0,0.017672
40401,39,1,52,9190754.0,184,291,82140,28094,1,0,0,1,0,0,0,0,0.00,0.0,0.095589
97921,19,0,25,9831531.0,210,324,40404,22335,0,1,0,0,0,1,0,0,0.00,0.0,0.013065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35736,15,1,19,6654598.0,101,178,32141,33939,0,1,0,1,0,0,0,0,0.00,0.0,0.252652
29419,38,0,23,10315211.0,184,304,83249,30460,0,1,0,0,1,0,0,0,0.00,0.0,0.252652
95255,15,0,22,1894435.0,61,83,10125,3760,1,0,0,0,0,1,0,0,0.00,0.0,0.252652
40257,56,0,18,2112065.0,21,46,65237,17302,1,0,0,0,0,1,0,0,0.00,0.0,0.252652


In [46]:
fraud_data.groupby('class')[['interval_after_signup']].mean()
#interval_after_signup on frauds are significantly lower compared to legits

,interval_after_signup
class,
0,5.191179e+06
1,2.570226e+06


In [47]:
fraud_data.groupby('class')[['interval_after_signup']].median() #1
# more than half of fraud happened 1s after signed up

,interval_after_signup
class,
0,5194911.0
1,1.0


In [48]:
fraud_data[fraud_data['class'] == 1].head(100)

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_day_of_year,purchase_day_of_year,signup_seconds_of_day,puchase_seconds_of_day
136961,24,VLHGCDPFCICDA,SEO,Chrome,F,33,3.432126e+09,1,United States,3327952.0,218,257,80113,38465
136962,14,YLUQSRNYYIPXU,Ads,Chrome,M,40,3.905319e+09,1,NA,1.0,12,12,4207,4208
136963,63,ABUBCQDATQMQH,Ads,FireFox,F,46,5.505670e+08,1,United States,7640070.0,49,137,40723,77593
136964,34,QHEODGCAVJKIQ,SEO,Chrome,M,37,9.408096e+08,1,United States,1.0,12,12,77710,77711
136965,76,DAKVYHKIEYRBH,SEO,Chrome,F,48,6.361041e+08,1,Hungary,1.0,10,10,48421,48422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137056,21,AIGPGDVRDKOKT,SEO,FireFox,F,45,3.058136e+09,1,India,1.0,2,2,69762,69763
137057,52,PQGKGQACIARBV,Ads,Chrome,F,36,7.203338e+08,1,China,1.0,9,9,77018,77019
137058,54,XHZBVWFWHSGTQ,SEO,FireFox,M,50,2.249217e+09,1,United States,1.0,1,1,32483,32484
137059,10,WETYPHOQVLWMK,Ads,FireFox,M,36,2.293333e+09,1,United States,1.0,7,7,63914,63915


### ***How to use predictions***

In [49]:
probsBest_recall

array([[0.4268741 , 0.5731259 ],
       [0.69920979, 0.30079021],
       [0.73063236, 0.26936764],
       ...,
       [0.73415242, 0.26584758],
       [0.7168678 , 0.2831322 ],
       [0.73652927, 0.26347073]])

In [50]:
probsBest_recall[:,1]

array([0.5731259 , 0.30079021, 0.26936764, ..., 0.26584758, 0.2831322 ,
       0.26347073])

In [51]:
# use Recall Score from the best Random Forest Model
# probsBest_recall[:,1] 选择第二列，lable是 Fraud 的概率

t = (10 * probsBest_recall[:,1]).astype(int)
unique, counts = np.unique(t, return_counts = True)

print(np.asarray((unique, counts)).T)

#recommended:
#green: 1 - 3 pass
#grey: 4-7 need manual investigation
#red: 8,9 decline

[[    2 19468]
 [    3  7731]
 [    4    92]
 [    5   192]
 [    6    32]
 [    7     2]
 [    8    38]
 [    9   121]]


### **Gradient Boosting**

In [0]:
from sklearn import ensemble


params = {'n_estimators': 500, 'max_depth': 3, 'subsample': 0.5,
          'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': 3}

gb_clf = ensemble.GradientBoostingClassifier(**params)
gb_clf.fit(X_train_sm, y_train_sm)


# make predictions
gb_y_pred = gb_clf.predict(X_test)

In [53]:

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, gb_y_pred)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1]))
print('precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1]))

confusion_matrix_sm is: 
        pred_0  pred_1
true_0   27380       7
true_1     198      91
recall_sm = 0.314878892733564
precision_sm = 0.9285714285714286


Optimizing recall score on Gradient Boost

In [0]:
param = {
    'max_depth':[None, 5, 15],
    'n_estimators':[10,150],
    'subsample':[0.7,0.9],
    'max_features':[None,3,6]
    }

gradientboost = ensemble.GradientBoostingClassifier(random_state=42)

In [67]:
grid_search_gb_recall = grid_search_wrapper(gradientboost,param, refit_score='recall_score')

Best parameters for recall_score
{'max_depth': None, 'max_features': None, 'n_estimators': 10, 'subsample': 0.7}

Confusion matrix of Random Forest optimized for recall_score on the test data:
        pred_0  pred_1
true_0   27125     262
true_1     128     161
	roc_auc_score is: : 0.7883468440245751
	f1_score is: : 0.45224719101123595
recall = 0.5570934256055363
precision = 0.3806146572104019


In [68]:
best_gb_model_recall = grid_search_gb_recall.best_estimator_
best_gb_model_recall

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=None,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

### **XGBoost**


Not using SMOTEd data set

In [0]:
import xgboost 
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

In [55]:
kfold = KFold(n_splits=10, random_state=42)
xgb_clf = XGBClassifier(objective='reg:logistic')
cv_results = cross_val_score(xgb_clf, X_train, y_train, cv=kfold, scoring = 'recall')
print('XGB ROCAUC on validation data: {0:.5f} {1:.5f}'.format(cv_results.mean(), cv_results.std()))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


XGB ROCAUC on validation data: 0.51085 0.03937


In [56]:
xgb_clf.fit(X_train, y_train, eval_metric='auc', verbose=False)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='reg:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [57]:
# make prediction on test set
xgb_y_pred = xgb_clf.predict(X_test)

# generate probability
xgb_probs = xgb_clf.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, xgb_y_pred)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, xgb_probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, xgb_y_pred)))#string to int

accuracy_score is: : 0.9953750541985836
roc_auc_score is: : 0.7922898154770727
f1_score is: : 0.7155555555555556


In [58]:

print ("confusion_matrix is: ")
cm_sm = confusion_matrix(y_test, xgb_y_pred)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1]))
print('precision =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1]))

confusion_matrix is: 
        pred_0  pred_1
true_0   27387       0
true_1     128     161
recall = 0.5570934256055363
precision = 1.0


In [59]:
xgb_probs

array([[0.97459173, 0.02540827],
       [0.99550295, 0.00449705],
       [0.9964283 , 0.00357167],
       ...,
       [0.99653244, 0.00346757],
       [0.99656266, 0.00343736],
       [0.9958955 , 0.0041045 ]], dtype=float32)

In [60]:
xgb_probs[:,1]

array([0.02540827, 0.00449705, 0.00357167, ..., 0.00346757, 0.00343736,
       0.0041045 ], dtype=float32)

In [61]:
# use Recall Score from the XGBoost Model
# probsBest_recall[:,1] 选择第二列，lable是 Fraud 的概率

t = (10 * xgb_probs[:,1]).astype(int)
unique, counts = np.unique(t, return_counts = True)

print(np.asarray((unique, counts)).T)

#recommended:
#green: 1 - 3 pass
#grey: 4-7 need manual investigation
#red: 8,9 decline

[[    0 27515]
 [    9   161]]
